# data読み込み

## read processed data

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pandas as pd
import time
import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy
from datetime import date, timedelta
import networkx as nx
import seaborn as sns
import pickle
import pandas.tseries.offsets as offsets
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
DATA_PATH = "../data/"
FILE_NAME= "capitaltrafficplus_202112.csv.gz"
df=pd.read_csv(DATA_PATH+FILE_NAME)

In [3]:
df

,linkid,gps_timestamp,speed_typea,accident_flag,real_accident_flag,weekdaytime,speed_typea_y
0,83585420,2021-12-01 00:00:00,73.075286,0,0,0.285998,60.062440
1,83585420,2021-12-01 00:10:00,79.817778,0,0,0.286991,64.814282
2,83585420,2021-12-01 00:20:00,52.938966,0,0,0.287984,70.731949
3,83585420,2021-12-01 00:30:00,71.913333,0,0,0.288977,100.642405
4,83585420,2021-12-01 00:40:00,71.913333,0,0,0.289970,101.989167
...,...,...,...,...,...,...,...
12682219,118770567,2021-12-31 23:10:00,53.953636,0,0,0.710030,46.808690
12682220,118770567,2021-12-31 23:20:00,45.952163,0,0,0.711023,47.108472
12682221,118770567,2021-12-31 23:30:00,45.952163,0,0,0.712016,54.059247
12682222,118770567,2021-12-31 23:40:00,38.102747,0,0,0.713009,48.534444


In [4]:
df["gps_timestamp"]=pd.to_datetime(df["gps_timestamp"])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12682224 entries, 0 to 12682223
Data columns (total 7 columns):
 #   Column              Dtype         
---  ------              -----         
 0   linkid              int64         
 1   gps_timestamp       datetime64[ns]
 2   speed_typea         float64       
 3   accident_flag       int64         
 4   real_accident_flag  int64         
 5   weekdaytime         float64       
 6   speed_typea_y       float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 677.3 MB


## read link connect relation

In [6]:
LINK_PATH = "../data/"
LINK_FILE_NAME= "capital_01_relation.csv"
link_df=pd.read_csv(LINK_PATH+LINK_FILE_NAME)
link_df=link_df.rename(columns={"Unnamed: 0":"row"})

In [7]:
link_df

,row,83585420,83585485,83585609,83585658,83585806,83585863,83585888,83586020,83586069,...,117489955,117490052,117490417,117490530,117590330,117632419,118415091,118415100,118770411,118770567
0,83585420,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,83585485,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,83585609,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,83585658,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,83585806,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836,117632419,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2837,118415091,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2838,118415100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2839,118770411,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
link_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2841 entries, 0 to 2840
Columns: 2842 entries, row to 118770567
dtypes: int64(2842)
memory usage: 61.6 MB


In [9]:
link_df.dtypes

row          int64
83585420     int64
83585485     int64
83585609     int64
83585658     int64
             ...  
117632419    int64
118415091    int64
118415100    int64
118770411    int64
118770567    int64
Length: 2842, dtype: object

# link単位車速可視化

## 事故が発生日＆link抽出

In [10]:
real_accident_links_df=df[df["real_accident_flag"]==1][["linkid","gps_timestamp"]]

In [11]:
real_accident_links_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4928 entries, 12248 to 12668194
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   linkid         4928 non-null   int64         
 1   gps_timestamp  4928 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 115.5 KB


In [12]:
real_accident_links_df

,linkid,gps_timestamp
12248,83585609,2021-12-24 01:20:00
12249,83585609,2021-12-24 01:30:00
12250,83585609,2021-12-24 01:40:00
50464,83586355,2021-12-10 10:40:00
50465,83586355,2021-12-10 10:50:00
...,...,...
12667577,118415091,2021-12-23 06:50:00
12667578,118415091,2021-12-23 07:00:00
12668192,118415091,2021-12-27 13:20:00
12668193,118415091,2021-12-27 13:30:00


In [13]:
real_accident_links_df["gps_timestamp"]=pd.to_datetime(real_accident_links_df["gps_timestamp"])
real_accident_links_df["gps_date"]=real_accident_links_df['gps_timestamp'].dt.strftime("%Y-%m-%d")
real_accident_links_df=real_accident_links_df.drop_duplicates(subset=["linkid","gps_date"],keep="first")
real_accident_links_df=real_accident_links_df.drop("gps_timestamp",axis=1)
display(real_accident_links_df)

,linkid,gps_date
12248,83585609,2021-12-24
50464,83586355,2021-12-10
73808,83586713,2021-12-17
79573,83586866,2021-12-26
94288,83587206,2021-12-04
...,...,...
12596469,116303066,2021-12-25
12610580,116353749,2021-12-30
12615045,116354815,2021-12-30
12667570,118415091,2021-12-23


## 個別車速可視化

In [14]:
vis_folder="./202112_real_accident_single_link/"

if not os.path.exists(vis_folder):
    os.makedirs(vis_folder)
meanful_real_accident_list=[]

def link_speed(linkid, date, df,meanful_real_accident_list,speed_diff_thres=20):
#     print(date,type(date))
#     print(datetime.datetime.strptime(date,'%Y-%m-%d'),type(datetime.datetime.strptime(date,'%Y-%m-%d')))
    datetime_date=datetime.datetime.strptime(date,'%Y-%m-%d')
    datetime_date2=datetime_date + timedelta(days=1)
    real_accident_link_date_df=df[df["linkid"]==linkid]
    real_accident_link_date_df=real_accident_link_date_df[(real_accident_link_date_df["gps_timestamp"]>=datetime_date) & (real_accident_link_date_df["gps_timestamp"]<datetime_date2)]
    #display(real_accident_link_date_df)
    
    accident_date=list(real_accident_link_date_df[real_accident_link_date_df["real_accident_flag"]==1]["gps_timestamp"])
    start_datetime = accident_date[0]
    end_datetime = accident_date[-1]
    
    speed_distance_list=[]
    speed_list=[]
    for date in accident_date:
        speed_distance_list.append(abs(real_accident_link_date_df[real_accident_link_date_df["gps_timestamp"]==date]["speed_typea_y"].values[0]-real_accident_link_date_df[real_accident_link_date_df["gps_timestamp"]==date]["speed_typea"].values[0]))
        speed_list.append(real_accident_link_date_df[real_accident_link_date_df["gps_timestamp"]==date]["speed_typea"].values[0])
    mean=sum(speed_distance_list)/len(speed_distance_list)
    speed_mean=sum(speed_list)/len(speed_list)
    if mean>=speed_diff_thres and len(accident_date)>3 and speed_mean<=40: #at least 30min  平均速度は60以下
        meanful_real_accident_list.append((linkid, date)) 
        #visualization
        plt.figure(figsize =(20, 8)) 
        plt.plot(real_accident_link_date_df["gps_timestamp"],real_accident_link_date_df["speed_typea"],label="speed_typea")
        plt.plot(real_accident_link_date_df["gps_timestamp"],real_accident_link_date_df["speed_typea_y"],label="speed_typea_y")
        plt.axvspan(start_datetime, end_datetime, color="red", alpha=0.1)
        plt.xlabel('Time')
        plt.ylabel('speed')
        plt.legend()
        plt.title(f"link id {linkid} accident time {start_datetime} mean speed_diff {mean}", fontsize=12, y=1)
        plt.savefig(vis_folder+f"link id {linkid} accident time{start_datetime}")
        plt.close()
        #plt.show()    


In [15]:
for i in tqdm(range(len(real_accident_links_df))):
    link_speed(real_accident_links_df["linkid"].values[i],real_accident_links_df["gps_date"].values[i],df,meanful_real_accident_list)
print(meanful_real_accident_list[:2], len(meanful_real_accident_list))

  0%|          | 0/1091 [00:00<?, ?it/s]

[(83590553, Timestamp('2021-12-25 17:30:00')), (83592015, Timestamp('2021-12-31 03:50:00'))] 212
